# Proceso de ETL

In [1]:
# Bibliotecas que necesitaremos
import gzip # Usamos la biblioteca gzip para extraer los archivos
import json
import pandas as pd
import ast

En el siguiente cuaderno se realizara ETL correspondiente a los archivos proporcionados en el siguiente orden:

1. steam_games.json.gz
2. user_reviews.json.gz
3. users_items.json.gz

## 1. Descomprimir json.gz a json

In [2]:
# Extrayendo steam_games.json.gz

# abrimos steam_games.json.gz como archivo_comprimido en modo lectura ('r') y texto ('t') con codificación UTF-8
# luego abrimos el archivo steam_games.json como archivo_json en modo escritura ('w') con codificación UTF-8.
# Usamos la construcción 'with' para garantizar que el archivo se cierre automáticamente cuando se termine de usar
with gzip.open('dataset/steam_games.json.gz', 'rt', encoding='utf-8') as archivo_comprimido, open('dataset/steam_games.json', 'w', encoding='utf-8') as archivo_json:
    
    # Luego leemos el archivo_comprimido con .read() y lo escribimos en archivo_json
    archivo_json.write(archivo_comprimido.read())

# Repetimos este proceso con los otros archivos

In [3]:
# Extrayendo user_reviews.json.gz

with gzip.open('dataset/user_reviews.json.gz', 'rt', encoding='utf-8') as archivo_comprimido, open('dataset/user_reviews.json', 'w', encoding='utf-8') as archivo_json:
    archivo_json.write(archivo_comprimido.read())

In [4]:
# Extrayendo users_items.json.gz

with gzip.open('dataset/users_items.json.gz', 'rt', encoding='utf-8') as archivo_comprimido, open('dataset/users_items.json', 'w', encoding='utf-8') as archivo_json:
    archivo_json.write(archivo_comprimido.read())

## 2. Transformaremos los datos a un formato mas eficiente para su analisis

En esta parte cargaremos los datos a Dataframes para analizarlos y ver cual es el mejor tratamiento para cada archivo para posteriormente guardarlos en archivos parquet, para procesamiento, y archivos csv, para que podamos revisarlos y analizarlos de forma visual, el orden en el cual haremos esto sera el siguiente:
1. steam_games.json
2. user_reviews.json
3. users_items.json

### 2.1 Tratamiento de users_items.json

Usaremos la funcion ast.literal_eval del modulo ast para el tratamiento de los archivos users_items.json y user_reviews, debido a que los archivos vienen en formatos no validos json, como por ejemplo: en los archivos encontramos el uso de comillas simples ('), lo cual no es valido ya que un archivo json debe tener comillas dobles

In [4]:
# Se crea una lista vacía llamada items que se utilizará para almacenar los objetos JSON leídos del archivo.
items = []
# Se abre el archivo en modo lectura como 'f'
with open('dataset/users_items.json', 'r', encoding='utf-8') as f:
    #  Se itera, leyendo el archivo f, linea por linea con la funcion readlines() a través de todas las líneas del archivo usando el bucle 'for'
    for line in f.readlines():
        # Dentro del bucle, se utiliza la función ast.literal_eval del módulo ast para evaluar la línea actual como una expresión literal de Python.
        items.append(ast.literal_eval(line))
# Las lineas del archivo se han convertido en objetos python almacenados en la lista items que seran convertidas al dataframe df_user_items
df_user_items = pd.DataFrame(items)
# display df
df_user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Podemos observar elementos anidados en la columna 'items' por lo que los inspeccionaremos

In [6]:
# Inspeccionamos el primer elemento de la columna 'items'
df_user_items['items'][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

Al ver la forma de esta lista de diccionarios con elementos y llaves iguales decidimos crear 2 dataframes derivados del archivo users_items.json:

- El Primer dataframe ser user_items y solo tendra 4 columnas que son: steam_id', 'user_id', 'user_url', 'items_count, la columna user_url no esta siendo considerada dada las condiciones del proyecto

In [7]:
# Creamos el primer dataframe reordenando las columnas para mejor practicidad

user_items = df_user_items[['steam_id', 'user_id', 'items_count']]
user_items

,steam_id,user_id,items_count
0,76561197970982479,76561197970982479,277
1,76561198035864385,js41637,888
2,76561198007712555,evcentric,137
3,76561197963445855,Riot-Punch,328
4,76561198002099482,doctr,541
...,...,...,...
88305,76561198323066619,76561198323066619,22
88306,76561198326700687,76561198326700687,177
88307,76561198328759259,XxLaughingJackClown77xX,0
88308,76561198329548331,76561198329548331,7


In [8]:
# Exportando el nuevo DataFrame a un archivo CSV
user_items.to_csv('dataset/user_items.csv', index=False)

- El segundo dataframe seera user_items_list y solo tendra las columnas 'steam_id' y 'items', en donde luego haremos la expansión de los datos contenidos en 'items'creando 4 columnas adicionales 'item_id', 'item_name', 'playtime_forever' y 'playtime_2weeks' para cada diccionario con sus respectivo 'steam_id'

In [9]:
# Creamos el nuevo Dataframe
user_items_list = df_user_items[['steam_id', 'items']]
user_items_list

,steam_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,76561198035864385,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,76561198007712555,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,76561197963445855,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,76561198002099482,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...
88305,76561198323066619,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,76561198328759259,[]
88308,76561198329548331,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [10]:
# Creamos una lista para almacenar los datos expandidos
expanded_data = []

# Se inicia un bucle que itera a través de cada fila del DataFrame user_items_list.
for index, row in user_items_list.iterrows():
    # Se obtiene la columna 'items' de la fila actual y se almacena en la variable items_list
    items_list = row['items']
    
    # Si la lista de diccionarios en 'items' está vacía, agregamos la fila original a 'expanded_data' sin ningun cambio
    if not items_list:
        expanded_data.append(row.to_dict())
    else:
        # Si la lista de 'items' no está vacía, la expandimos, de tal forma que item_dict hace referencia a un diccionario dentro de la lista
        for item_dict in items_list:
            # Creamos un nuevo diccionario basado en la fila original
            new_data = row.to_dict()
            # Añadimos las columnas adicionales a partir de los datos del 'item_dict'
            new_data['item_id'] = item_dict['item_id']
            new_data['item_name'] = item_dict['item_name']
            new_data['playtime_forever'] = item_dict['playtime_forever']
            new_data['playtime_2weeks'] = item_dict['playtime_2weeks']
            # Agregamos el nuevo diccionario a 'expanded_data'
            expanded_data.append(new_data)

# Creamos un nuevo DataFrame a partir de los datos expandidos
user_items_list = pd.DataFrame(expanded_data)

user_items_list.drop('items', axis=1, inplace=True)

user_items_list

,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...
5170010,76561198329548331,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,519140,Minds Eyes,3.0,3.0


In [11]:
# Exportando el nuevo DataFrame a un archivo CSV
user_items_list.to_csv('dataset/user_items_list.csv', index=False)

### 2.2 Tratamiento de user_reviews.json

Usaremos un enfoque parecido al archivo users_items.json

In [2]:
# Se crea una lista vacía llamada items que se utilizará para almacenar los objetos JSON leídos del archivo.
reviews = []
# Se abre el archivo en modo lectura como 'f'
with open('dataset/user_reviews.json', 'r', encoding='utf-8') as f:
    #  Se itera, leyendo el archivo f, linea por linea con la funcion readlines() a través de todas las líneas del archivo usando el bucle 'for'
    for line in f.readlines():
        # Dentro del bucle, se utiliza la función ast.literal_eval del módulo ast para evaluar la línea actual como una expresión literal de Python.
        reviews.append(ast.literal_eval(line))
# Las lineas del archivo se han convertido en objetos python almacenados en la lista items que seran convertidas al dataframe df_user_items
df_user_reviews = pd.DataFrame(reviews)
# display df
df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Vemos una situacion parecida al anterior archivo en donde una columna esta anidada, ademas debemos tener en cuenta que de preferencia lo mejor es tener un identificador id para referenciar a un user_id, es por ello que traeremos de la tabla user_items el steam_id para luego eliminar las columnas user_id y user_url

In [5]:
# Realiza una fusión (merge) basada en la columna 'user_id' para agregar 'steam_id' a df_user_reviews
df_user_reviews = df_user_reviews.merge(df_user_items[['user_id', 'steam_id']], on='user_id', how='left')
df_user_reviews

,user_id,user_url,reviews,steam_id
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",76561197970982479
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",76561198035864385
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...",76561198007712555
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...",76561198002099482
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...",76561198026584251
...,...,...,...,...
26434,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la...",76561198306599751
26435,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l...",76561198308105615
26436,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',...",76561198310819422
26437,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l...",76561198312638244


In [6]:
df_user_reviews = df_user_reviews[['steam_id', 'reviews']]
df_user_reviews

,steam_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,76561198035864385,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,76561198007712555,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,76561198002099482,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,76561198026584251,"[{'funny': '3 people found this review funny',..."
...,...,...
26434,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
26435,76561198308105615,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
26436,76561198310819422,"[{'funny': '1 person found this review funny',..."
26437,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [7]:
# Ahora observamos un elemento de la columna 'reviews para observar como se comporta'
df_user_reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Observamos que se da un fenomeno similar anterior archivo por lo que tambien expandiremos este Dataframe

In [8]:
# Creamos una lista para almacenar los datos expandidos
expanded_data = []

# Iteramos sobre cada fila del DataFrame original ('prueba')
for index, row in df_user_reviews.iterrows():
    # Obtenemos la lista de diccionarios en la columna 'items' de la fila actual
    items_list = row['reviews']
    
    # Si la lista de 'items' está vacía, agregamos la fila original a 'expanded_data'
    if not items_list:
        expanded_data.append(row.to_dict())
    else:
        # Si la lista de 'items' no está vacía, la expandimos
        for item_dict in items_list:
            # Creamos un nuevo diccionario basado en la fila original
            new_data = row.to_dict()
            # Añadimos las columnas adicionales a partir de los datos del 'item_dict'
            new_data['funny'] = item_dict['funny']
            new_data['posted'] = item_dict['posted']
            new_data['item_id'] = item_dict['item_id']
            new_data['helpful'] = item_dict['helpful']
            new_data['recommend'] = item_dict['recommend']
            new_data['review'] = item_dict['review']
            # Agregamos el nuevo diccionario a 'expanded_data'
            expanded_data.append(new_data)

# Creamos un nuevo DataFrame a partir de los datos expandidos
user_reviews = pd.DataFrame(expanded_data)

user_reviews.drop('reviews', axis=1, inplace=True)

user_reviews

,steam_id,funny,posted,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,76561198035864385,,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,76561198035864385,,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
61104,76561198312638244,,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
61105,76561198312638244,,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
61106,76561198313816521,1 person found this review funny,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
61107,76561198313816521,,Posted July 20.,730,No ratings yet,True,:D


In [9]:
# Reordenamos y quitamos las columnas helpful y funny
user_reviews = user_reviews[['steam_id', 'item_id', 'posted', 'recommend','review']]
user_reviews

,steam_id,item_id,posted,recommend,review
0,76561197970982479,1250,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,76561197970982479,43110,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,76561198035864385,251610,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,76561198035864385,227300,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
61104,76561198312638244,70,Posted July 10.,True,a must have classic from steam definitely wort...
61105,76561198312638244,362890,Posted July 8.,True,this game is a perfect remake of the original ...
61106,76561198313816521,273110,Posted July 3.,True,had so much fun plaing this and collecting res...
61107,76561198313816521,730,Posted July 20.,True,:D


In [10]:
# Tenemos que guardar steam_id en formato int64 para evitar la perdida de información
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61109 entries, 0 to 61108
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steam_id   61094 non-null  object
 1   item_id    61080 non-null  object
 2   posted     61080 non-null  object
 3   recommend  61080 non-null  object
 4   review     61080 non-null  object
dtypes: object(5)
memory usage: 2.3+ MB


In [12]:
user_reviews = user_reviews.dropna(subset=["steam_id"]).reset_index(drop=True)
user_reviews

,steam_id,item_id,posted,recommend,review
0,76561197970982479,1250,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,76561197970982479,43110,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,76561198035864385,251610,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,76561198035864385,227300,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
61089,76561198312638244,70,Posted July 10.,True,a must have classic from steam definitely wort...
61090,76561198312638244,362890,Posted July 8.,True,this game is a perfect remake of the original ...
61091,76561198313816521,273110,Posted July 3.,True,had so much fun plaing this and collecting res...
61092,76561198313816521,730,Posted July 20.,True,:D


In [13]:
user_reviews['steam_id'] = user_reviews['steam_id'].astype('int64')

In [14]:
# Exportando el nuevo DataFrame a un archivo CSV
user_reviews.to_csv('dataset/user_reviews.csv', index=False)

### 2.3 Tratamiento de steam_games.json

Para el tratamiento de este archivo podemos usar read_json puesto que este si entra en la categoria de json valido por lo que no habra mas complicaciones

In [57]:
steam_games = pd.read_json('dataset/steam_games.json', lines=True)
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


Tambien para este caso al hacer una inspeccion del archivo observamos una gran cantidad de filas vacias por lo que para este caso eliminaremos todas estas.

In [58]:
# Elimina las filas en las que todos los valores son "None" y resetea el índice
steam_games = steam_games.dropna(how='all').reset_index(drop=True)
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
4,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [59]:
# Quitamos columnas innecesarias y reordenamos
steam_games = steam_games[['id', 'title', 'url', 'app_name', 'genres', 'developer', 'publisher', 'release_date', 'price']]
steam_games

,id,title,url,app_name,genres,developer,publisher,release_date,price
0,761140.0,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",Kotoshiro,Kotoshiro,2018-01-04,4.99
1,643980.0,Ironbound,http://store.steampowered.com/app/643980/Ironb...,Ironbound,"[Free to Play, Indie, RPG, Strategy]",Secret Level SRL,"Making Fun, Inc.",2018-01-04,Free To Play
2,670290.0,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",Poolians.com,Poolians.com,2017-07-24,Free to Play
3,767400.0,弹炸人2222,http://store.steampowered.com/app/767400/2222/,弹炸人2222,"[Action, Adventure, Casual]",彼岸领域,彼岸领域,2017-12-07,0.99
4,773570.0,None,http://store.steampowered.com/app/773570/Log_C...,Log Challenge,None,None,None,None,2.99
...,...,...,...,...,...,...,...,...,...
32130,773640.0,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,Colony On Mars,"[Casual, Indie, Simulation, Strategy]","Nikita ""Ghost_RUS""",Ghost_RUS Games,2018-01-04,1.99
32131,733530.0,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",Sacada,Sacada,2018-01-04,4.99
32132,610660.0,Russian Roads,http://store.steampowered.com/app/610660/Russi...,Russian Roads,"[Indie, Racing, Simulation]",Laush Dmitriy Sergeevich,Laush Studio,2018-01-04,1.99
32133,658870.0,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,EXIT 2 - Directions,"[Casual, Indie]","xropi,stev3ns",SIXNAILS,2017-09-02,4.99


Podemos observar que la columna id hace referencia a item_id de nuestras anteriores tablas, por lo que para mejor legibilidad y tratamiento cambiaremos id a item_id y title a item_name

In [60]:
steam_games.rename(columns={'id': 'item_id', 'title': 'item_name'}, inplace=True)
steam_games

C:\Users\ERWIN\AppData\Local\Temp\ipykernel_68880\2691029697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games.rename(columns={'id': 'item_id', 'title': 'item_name'}, inplace=True)


,item_id,item_name,url,app_name,genres,developer,publisher,release_date,price
0,761140.0,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",Kotoshiro,Kotoshiro,2018-01-04,4.99
1,643980.0,Ironbound,http://store.steampowered.com/app/643980/Ironb...,Ironbound,"[Free to Play, Indie, RPG, Strategy]",Secret Level SRL,"Making Fun, Inc.",2018-01-04,Free To Play
2,670290.0,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",Poolians.com,Poolians.com,2017-07-24,Free to Play
3,767400.0,弹炸人2222,http://store.steampowered.com/app/767400/2222/,弹炸人2222,"[Action, Adventure, Casual]",彼岸领域,彼岸领域,2017-12-07,0.99
4,773570.0,None,http://store.steampowered.com/app/773570/Log_C...,Log Challenge,None,None,None,None,2.99
...,...,...,...,...,...,...,...,...,...
32130,773640.0,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,Colony On Mars,"[Casual, Indie, Simulation, Strategy]","Nikita ""Ghost_RUS""",Ghost_RUS Games,2018-01-04,1.99
32131,733530.0,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",Sacada,Sacada,2018-01-04,4.99
32132,610660.0,Russian Roads,http://store.steampowered.com/app/610660/Russi...,Russian Roads,"[Indie, Racing, Simulation]",Laush Dmitriy Sergeevich,Laush Studio,2018-01-04,1.99
32133,658870.0,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,EXIT 2 - Directions,"[Casual, Indie]","xropi,stev3ns",SIXNAILS,2017-09-02,4.99


Cambiaremos item_id a tipo int por lo que tenemos que verificar si tiene elementos tipo None o NaN

In [61]:
invalid_item_id_rows = steam_games.loc[steam_games['item_id'].isna()]
invalid_item_id_rows

,item_id,item_name,url,app_name,genres,developer,publisher,release_date,price
74,NaN,None,http://store.steampowered.com/,None,None,None,None,None,19.99
30961,NaN,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,Batman: Arkham City - Game of the Year Edition,"[Action, Adventure]","Rocksteady Studios,Feral Interactive (Mac)","Warner Bros. Interactive Entertainment, Feral ...",2012-09-07,19.99


Eliminaremos la fila 74 y de la url se recuperara el id faltante

In [63]:
# Elimina la fila en el índice 74
steam_games = steam_games.drop(74)

# Restablece el índice
steam_games = steam_games.reset_index(drop=True)

# Reemplaza los valores NaN por un valor predeterminado
steam_games['item_id'].fillna(200260, inplace=True)

# Convierte la columna 'item_id' a tipo de datos entero
steam_games['item_id'] = steam_games['item_id'].astype(int)
#Eliminamos la columna 'url'
steam_games = steam_games[['item_id', 'item_name', 'app_name', 'genres', 'developer', 'publisher', 'release_date', 'price']]
steam_games

,item_id,item_name,app_name,genres,developer,publisher,release_date,price
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",Kotoshiro,Kotoshiro,2018-01-04,4.99
1,643980,Ironbound,Ironbound,"[Free to Play, Indie, RPG, Strategy]",Secret Level SRL,"Making Fun, Inc.",2018-01-04,Free To Play
2,670290,Real Pool 3D - Poolians,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",Poolians.com,Poolians.com,2017-07-24,Free to Play
3,767400,弹炸人2222,弹炸人2222,"[Action, Adventure, Casual]",彼岸领域,彼岸领域,2017-12-07,0.99
4,773570,None,Log Challenge,None,None,None,None,2.99
...,...,...,...,...,...,...,...,...
32128,773640,Colony On Mars,Colony On Mars,"[Casual, Indie, Simulation, Strategy]","Nikita ""Ghost_RUS""",Ghost_RUS Games,2018-01-04,1.99
32129,733530,LOGistICAL: South Africa,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",Sacada,Sacada,2018-01-04,4.99
32130,610660,Russian Roads,Russian Roads,"[Indie, Racing, Simulation]",Laush Dmitriy Sergeevich,Laush Studio,2018-01-04,1.99
32131,658870,EXIT 2 - Directions,EXIT 2 - Directions,"[Casual, Indie]","xropi,stev3ns",SIXNAILS,2017-09-02,4.99


Observando la columna genres, vemos que generalmente esta conformada por una lista irregulas por lo que contaremos la cantidad de veces que aparece cada elemento para saber cual es la mejor forma de tratarla

In [65]:
# Inicializamos un diccionario para contar los géneros
genre_counts = {}

# Iteramos sobre las listas en la columna 'genres' teniendo en cuenta elementos None
for genres_list in steam_games['genres']:
    if genres_list is not None:
        for genre in genres_list:
            # Si el género ya está en el diccionario, aumentamos el contador en 1
            if genre in genre_counts:
                genre_counts[genre] += 1
            # Si el género no está en el diccionario, lo agregamos con un contador de 1
            else:
                genre_counts[genre] = 1

# Convertimos el diccionario a una Serie para mostrar los resultados
genre_counts_series = pd.Series(genre_counts)

# Ahora 'genre_counts_series' contiene los géneros y sus recuentos, considerando elementos None
genre_counts_series

Action                       11321
Casual                        8282
Indie                        15858
Simulation                    6699
Strategy                      6956
Free to Play                  2031
RPG                           5479
Sports                        1257
Adventure                     8243
Racing                        1083
Early Access                  1462
Massively Multiplayer         1108
Animation &amp; Modeling       183
Video Production               116
Utilities                      340
Web Publishing                 268
Education                      125
Software Training              105
Design &amp; Illustration      460
Audio Production                93
Photo Editing                   77
Accounting                       7
dtype: int64

Observando la gran cantidad de generos lo mejor es crear 2 Dataframes para un mejor analisis:

- El primer Dataframe tendra las mismas columnas pero no contara con la columna genres

In [66]:
#Eliminamos la columna genres
steam_games1 = steam_games.drop(columns='genres')
steam_games1

,item_id,item_name,app_name,developer,publisher,release_date,price
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro,Kotoshiro,2018-01-04,4.99
1,643980,Ironbound,Ironbound,Secret Level SRL,"Making Fun, Inc.",2018-01-04,Free To Play
2,670290,Real Pool 3D - Poolians,Real Pool 3D - Poolians,Poolians.com,Poolians.com,2017-07-24,Free to Play
3,767400,弹炸人2222,弹炸人2222,彼岸领域,彼岸领域,2017-12-07,0.99
4,773570,None,Log Challenge,None,None,None,2.99
...,...,...,...,...,...,...,...
32128,773640,Colony On Mars,Colony On Mars,"Nikita ""Ghost_RUS""",Ghost_RUS Games,2018-01-04,1.99
32129,733530,LOGistICAL: South Africa,LOGistICAL: South Africa,Sacada,Sacada,2018-01-04,4.99
32130,610660,Russian Roads,Russian Roads,Laush Dmitriy Sergeevich,Laush Studio,2018-01-04,1.99
32131,658870,EXIT 2 - Directions,EXIT 2 - Directions,"xropi,stev3ns",SIXNAILS,2017-09-02,4.99


In [70]:
# Exportando el nuevo DataFrame a un archivo CSV
steam_games1.to_csv('dataset/steam_games.csv', index=False)

- Para el segundo Dataframe dejaremos por el momento nos quedaremos con el id y y con genres

In [69]:
steam_games2 = steam_games[['item_id', 'genres']]
steam_games2

,item_id,genres
0,761140,"[Action, Casual, Indie, Simulation, Strategy]"
1,643980,"[Free to Play, Indie, RPG, Strategy]"
2,670290,"[Casual, Free to Play, Indie, Simulation, Sports]"
3,767400,"[Action, Adventure, Casual]"
4,773570,None
...,...,...
32128,773640,"[Casual, Indie, Simulation, Strategy]"
32129,733530,"[Casual, Indie, Strategy]"
32130,610660,"[Indie, Racing, Simulation]"
32131,658870,"[Casual, Indie]"


In [71]:
# Primero, explota la columna 'genres' para convertir las listas en filas separadas
steam_games3 = steam_games2.explode('genres')
steam_games3

,item_id,genres
0,761140,Action
0,761140,Casual
0,761140,Indie
0,761140,Simulation
0,761140,Strategy
...,...,...
32130,610660,Racing
32130,610660,Simulation
32131,658870,Casual
32131,658870,Indie


In [72]:
# Exportando el nuevo DataFrame a un archivo CSV
steam_games3.to_csv('dataset/steam_games_genres.csv', index=False)